### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, Model, layers
import numpy as np 
import matplotlib.pyplot as plt
import json
import os 
from PIL import Image
import pandas as pd

### TFrec file & batch size

In [ ]:
IDs = ['00503','01046','01231','01816','01866','02015','02152','02459','03004','03253']

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

TEST_FILENAMES = '../../datasets/Users_google_tfrec/'+ IDs[9]+'.tfrec'
# TEST_FILENAMES = '../../datasets/mit_split_tfrec/test.tfrec'
BATCH_SIZE = 1

SEED = tf.Variable(256)

### Parse TFRec fn & Transforms

In [ ]:
def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "path": tf.io.FixedLenFeature([], tf.string),
        "device": tf.io.FixedLenFeature([], tf.string),
        "screen_h": tf.io.FixedLenFeature([], tf.int64),
        "screen_w": tf.io.FixedLenFeature([], tf.int64),
        "face_valid": tf.io.FixedLenFeature([], tf.int64),
        "face_x": tf.io.FixedLenFeature([], tf.int64),
        "face_y": tf.io.FixedLenFeature([], tf.int64),
        "face_w": tf.io.FixedLenFeature([], tf.int64),
        "face_h": tf.io.FixedLenFeature([], tf.int64),
        "leye_x": tf.io.FixedLenFeature([], tf.int64),
        "leye_y": tf.io.FixedLenFeature([], tf.int64),
        "leye_w": tf.io.FixedLenFeature([], tf.int64),
        "leye_h": tf.io.FixedLenFeature([], tf.int64),
        "reye_x": tf.io.FixedLenFeature([], tf.int64),
        "reye_y": tf.io.FixedLenFeature([], tf.int64),
        "reye_w": tf.io.FixedLenFeature([], tf.int64),
        "reye_h": tf.io.FixedLenFeature([], tf.int64),
        "dot_xcam": tf.io.FixedLenFeature([], tf.float32),
        "dot_y_cam": tf.io.FixedLenFeature([], tf.float32),
        "dot_x_pix": tf.io.FixedLenFeature([], tf.float32),
        "dot_y_pix": tf.io.FixedLenFeature([], tf.float32),
        "reye_x1": tf.io.FixedLenFeature([], tf.int64),
        "reye_y1": tf.io.FixedLenFeature([], tf.int64),
        "reye_x2": tf.io.FixedLenFeature([], tf.int64),
        "reye_y2": tf.io.FixedLenFeature([], tf.int64),
        "leye_x1": tf.io.FixedLenFeature([], tf.int64),
        "leye_y1": tf.io.FixedLenFeature([], tf.int64),
        "leye_x2": tf.io.FixedLenFeature([], tf.int64),
        "leye_y2": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    return example


def augmentation(image, training = True):
    if training:
        aug = tf.keras.Sequential([
                layers.Resizing(128+10, 128+10),
                layers.RandomCrop(128, 128, 256),
                layers.Rescaling(1./255),
                layers.Normalization(mean = (0.3741, 0.4076, 0.5425), variance = (0.0004, 0.0004, 0.0004))
                ])
        
    else:
        aug = tf.keras.Sequential([
                layers.Resizing(128, 128),
                layers.Rescaling(1./255),
                layers.Normalization(mean = (0.3741, 0.4076, 0.5425), variance = (0.0004, 0.0004, 0.0004))
                ])
    
    image = aug(image)
    
    return image

def prepare_eval_sample(features):
    img_feat = features['image']

    h = tf.shape(img_feat)[0]
    w = tf.shape(img_feat)[1]

    w = tf.cast(w, tf.int64)
    h = tf.cast(h, tf.int64)

    screen_w, screen_h = features['screen_w'], features['screen_h']

    kps = [features['leye_x1']/w, features['leye_y1']/h, features['leye_x2']/w, features['leye_y2']/h,
           features['reye_x1']/w, features['reye_y1']/h, features['reye_x2']/w, features['reye_y2']/h]
    # kps has type float64

    lx, ly, lw, lh = int(features['leye_x']), int(features['leye_y']), int(features['leye_w']), int(features['leye_h'])
    rx, ry, rw, rh = int(features['reye_x']), int(features['reye_y']), int(features['reye_w']), int(features['reye_h'])

    lx = tf.clip_by_value(lx, 0, int(w)-lw)
    ly = tf.clip_by_value(ly, 0, int(h)-lh)

    rx = tf.clip_by_value(rx, 0, int(w)-rw)
    ry = tf.clip_by_value(ry, 0, int(h)-rh)

    l_eye = tf.image.crop_to_bounding_box(img_feat, ly, lx, lh, lw)
    r_eye = tf.image.crop_to_bounding_box(img_feat, ry, rx, rh, rw)

    l_eye = tf.image.flip_left_right(l_eye)

    l_eye = augmentation(l_eye, False)
    r_eye = augmentation(r_eye, False)

    y = [features['dot_xcam'], features['dot_y_cam']]
    # y has type float32
    
    path = features['path']

    return (l_eye, r_eye, kps), y, path

def get_eval_dataset(filenames, batch_size):
    option_no_order = tf.data.Options()
    option_no_order.deterministic = False  # disable order, increase speed
    
    dataset = (
        tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
        .with_options(option_no_order)
        .map(parse_tfrecord_fn, num_parallel_calls=AUTO)
        .map(prepare_eval_sample, num_parallel_calls=AUTO)
        .batch(batch_size)
    )
    
    dataset_len = sum(1 for _ in tf.data.TFRecordDataset(filenames))
    print(f"No. of samples: {dataset_len}")
    
    return dataset

### Create dataloader

In [ ]:
individual_dataset = get_eval_dataset(TEST_FILENAMES, BATCH_SIZE)

## Model

In [ ]:
gs_model = tf.keras.models.load_model('../../checkpoints/new_mod/gs/rlrop100/epoch-99-vl-2.371.ckpt')

### Model Summary

In [ ]:
gs_model.summary()

### Penultimate Layer

In [ ]:
layer_name = 're_lu_2'
pen_model = Model(inputs=gs_model.input, 
                  outputs=gs_model.get_layer(layer_name).output)

### Pen_model summary

In [ ]:
pen_model.summary()

## Preds

### tostr fn

In [ ]:
def tensor2str(path):
    path_np = np.array(path)
    path_str = np.array_str(path_np)
    result_1 = path_str.split('images/', 1)[1]
    image_id = result_1.split('.jpg', 1)[0]
    return image_id

### Get PU & GT

In [ ]:
data = []
for x_test,y_test, path in individual_dataset:
    image_id = tensor2str(path)
    pred = pen_model.predict(x_test, verbose=0) 
    pred_np = np.array(pred)
    target = np.array(y_test)
    
    data.append([image_id,[pred_np],[target]])

In [ ]:
pts = np.unique(target, axis=0)
len(pts)

### Checking Results format

In [ ]:
len(data), data[10]

In [ ]:
print(data[10][0])
print(data[10][1])
print(data[10][2])

## Create CSV

In [ ]:
df=pd.DataFrame(data,columns=['Image ID','Penultimate_Output','GT_Value'])
path_csv = '../../CSVs/google_all/'+IDs[9]+'.csv'
df.to_csv(path_csv, index = False)
df.head()